In [1]:
import os
# Set CUDA visible devices
os.environ['CUDA_VISIBLE_DEVICES'] = '0' #'-1'

In [2]:
# Check Pytorch installation
import torch, torchvision
print('Torch Version: ',torch.__version__)
print('Is Torch CUDA available? ', torch.cuda.is_available())

# Check MMDetection installation
import mmgen
print('MMGeneration Version: ', mmgen.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print('CUDA Compiler Version: ', get_compiling_cuda_version())
print('GCC Version: ',get_compiler_version())

Torch Version:  1.7.1+cu110
Is Torch CUDA available?  True


C:\Users\Ali\Anaconda3\envs\mmgen\lib\site-packages\mmcv\__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


MMGeneration Version:  0.7.3


ImportError: DLL load failed while importing _ext: The specified procedure could not be found.

In [3]:
import os
os.chdir('G:/mmgeneration')
os.listdir()

['.circleci',
 '.dev_scripts',
 '.git',
 '.github',
 '.gitignore',
 '.ipynb_checkpoints',
 '.pre-commit-config.yaml',
 '.pylintrc',
 '.readthedocs.yml',
 'apps',
 'build',
 'CITATION.cff',
 'configs',
 'demo',
 'docker',
 'docs',
 'LICENSE',
 'LICENSES.md',
 'MANIFEST.in',
 'mmcv_full-1.7.1-cp39-cp39-win_amd64.whl',
 'mmgen',
 'mmgen.egg-info',
 'model-index.yml',
 'output',
 'README.md',
 'README_zh-CN.md',
 'requirements',
 'requirements.txt',
 'setup.cfg',
 'setup.py',
 'stylegan2-church-config-f-official_20210327_172657-1d42b7d1.pth',
 'test run ali.ipynb',
 'test train ali.ipynb',
 'tests',
 'test_run',
 'tools']

In [4]:
import mmcv
from mmgen.apis import init_model, sample_unconditional_model

# Specify the path to model config and checkpoint file
# config_file = 'configs/styleganv2/stylegan2_c2_lsun-church_256_b4x8_800k.py'
config_file = 'configs/styleganv2/stylegan2_c2_places365_256_b4x8_800k_ali.py'
# you can download this checkpoint in advance and use a local file path.
checkpoint_file = 'stylegan2-church-config-f-official_20210327_172657-1d42b7d1.pth'

device = 'cuda:0' #'cpu' #'cuda:0'
model = init_model(config_file, checkpoint_file, device=device)

C:\Users\Ali\Anaconda3\envs\mmgen\lib\site-packages\mmcv\cnn\bricks\transformer.py:33: UserWarning: Fail to import ``MultiScaleDeformableAttention`` from ``mmcv.ops.multi_scale_deform_attn``, You should install ``mmcv-full`` if you need this module. 
  warnings.warn('Fail to import ``MultiScaleDeformableAttention`` from '


ImportError: DLL load failed while importing _ext: The specified procedure could not be found.

In [5]:
from mmgen.apis import set_random_seed


from mmcv import Config
cfg = Config.fromfile(config_file)


In [6]:
img_size = 256
# cfg.img_size = img_size
# cfg.train_pipeline[1].scale=(img_size,img_size)
# cfg.val_pipeline[1].scale=(img_size,img_size)
cfg.model.generator.out_size = img_size
cfg.model.discriminator.in_size = img_size

cfg.data.samples_per_gpu=1 #1
cfg.data.workers_per_gpu=1 #4

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

cfg.use_ddp_wrapper = True
cfg.find_unused_parameters = True  # True for dynamic model, False for static model
cfg.runner = dict(
    type='DynamicIterBasedRunner',
    is_dynamic_ddp=False,  # Note that this flag should be False.
    pass_training_status=True)


cfg.log_config.interval=1
# cfg.optimizer_config = dict(grad_clip=None)
cfg.workflow = [('train', 1)]

cfg.load_from = "stylegan2-church-config-f-official_20210327_172657-1d42b7d1.pth"
cfg.work_dir = './test_run'

In [7]:
print(f'Config:\n{cfg.pretty_text}')

Config:
dataset_type = 'UnconditionalImageDataset'
img_size = 256
train_pipeline = [
    dict(type='LoadImageFromFile', key='real_img', io_backend='disk'),
    dict(type='Resize', keys=['real_img'], scale=(256, 256)),
    dict(
        type='Normalize',
        keys=['real_img'],
        mean=[127.5, 127.5, 127.5],
        std=[127.5, 127.5, 127.5],
        to_rgb=False),
    dict(type='ImageToTensor', keys=['real_img']),
    dict(type='Collect', keys=['real_img'], meta_keys=['real_img_path'])
]
val_pipeline = [
    dict(type='LoadImageFromFile', key='real_img', io_backend='disk'),
    dict(type='Resize', keys=['real_img'], scale=(256, 256)),
    dict(
        type='Normalize',
        keys=['real_img'],
        mean=[127.5, 127.5, 127.5],
        std=[127.5, 127.5, 127.5],
        to_rgb=True),
    dict(type='ImageToTensor', keys=['real_img']),
    dict(type='Collect', keys=['real_img'], meta_keys=['real_img_path'])
]
data = dict(
    samples_per_gpu=1,
    workers_per_gpu=1,
    trai

In [8]:
from mmgen.datasets import build_dataset
from mmgen.models import build_model
from mmgen.apis import train_model
import os
import mmcv

# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build model
# model = build_model(cfg.model)

# Create work_dir
mmcv.mkdir_or_exist(os.path.abspath(cfg.work_dir))

2023-07-18 20:23:43,556 - mmgen - INFO - dataset_name: <class 'mmgen.datasets.unconditional_image_dataset.UnconditionalImageDataset'>, total 21900 images in imgs_root: G:/Places365_small/val_256/train_samples


In [9]:
train_model(model, datasets, cfg, distributed=False, validate=True)

2023-07-18 20:23:44,493 - mmgen - INFO - load checkpoint from local path: stylegan2-church-config-f-official_20210327_172657-1d42b7d1.pth
2023-07-18 20:23:44,742 - mmgen - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: generator_ema.injected_noise_0, generator_ema.injected_noise_1, generator_ema.injected_noise_2, generator_ema.injected_noise_3, generator_ema.injected_noise_4, generator_ema.injected_noise_5, generator_ema.injected_noise_6, generator_ema.injected_noise_7, generator_ema.injected_noise_8, generator_ema.injected_noise_9, generator_ema.injected_noise_10, generator_ema.injected_noise_11, generator_ema.injected_noise_12, generator_ema.style_mapping.1.bias, generator_ema.style_mapping.1.linear.weight_orig, generator_ema.style_mapping.2.bias, generator_ema.style_mapping.2.linear.weight_orig, generator_ema.style_mapping.3.bias, generator_ema.style_mapping.3.linear.weight_orig, generator_ema.style_mapping.4.bias, generator_ema.

RuntimeError: No such operator aten::cudnn_convolution_backward_weight

In [ ]:
# !python tools/train.py \
#     "configs/styleganv2/stylegan2_c2_lsun-church_256_b4x8_800k.py" \
#     --work-dir "./test_run/"